A quick notebook for getting SSIS execution

Connect to SSIS DB

In [ ]:
USE SSISDB
GO

-- Find all error messages
SELECT
    OM.operation_message_id
,   OM.operation_id
,   OM.message_time
,   OM.message_type
,   OM.message_source_type
,   OM.message
,   OM.extended_info_id
FROM
    catalog.operation_messages AS OM
WHERE
    OM.message_type = 120
ORDER BY Message_Time DESC

Get all failures

In [ ]:
USE SSISDB
GO
-- Generate all the messages associated to failing operations
SELECT
    OM.operation_message_id
,   OM.operation_id
,   OM.message_time
,   OM.message_type
,   OM.message_source_type
,   OM.message
,   OM.extended_info_id
FROM
    catalog.operation_messages AS OM
    INNER JOIN
    (  
        -- Find failing operations
        SELECT DISTINCT
            OM.operation_id  
        FROM
            catalog.operation_messages AS OM
        WHERE
            OM.message_type = 120
    ) D
    ON D.operation_id = OM.operation_id
WHERE message_time < DATEADD(Day,-7,GetDate())
ORDER BY message_time DESC

In [ ]:
-- Find all messages associated to the last failing run
USE SSISDB
GO
SELECT
    OM.operation_message_id
,   OM.operation_id
,   OM.message_time
,   OM.message_type
,   OM.message_source_type
,   OM.message
,   OM.extended_info_id
FROM
    catalog.operation_messages AS OM
WHERE
    OM.operation_id = 
    (  
        -- Find the last failing operation
        -- lazy assumption that biggest operation
        -- id is last. Could be incorrect if a long
        -- running process fails after a quick process
        -- has also failed
        SELECT 
            MAX(OM.operation_id)
        FROM
            catalog.operation_messages AS OM
        WHERE
            OM.message_type = 120
              );